In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os


In [2]:
###2.# Define the directory where your models are saved

parent_dir = os.path.dirname(os.getcwd())


model_directory = os.path.join(parent_dir, 'Step 3 External Validation and Model Development/Classification Models')

model_1_list = []
model_2_list = []
#,8,9,10,11,12,13,14,15,16,17,18,41
# Assuming your models are named 'model_0.h5', 'model_1.h5', ..., 'model_19.h5'
model1_paths = [os.path.join(model_directory, f'Complete Set 1 Models/ROBOMechDB Complete Set 1 Model Seed {i+1}.keras') for i in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,41]]

model2_paths = [os.path.join(model_directory, f'Complete Set 2 Models/ROBOMechDB Complete Set 2 Model Seed {i+1}.keras') for i in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,41]]


for path in model1_paths:
    print(f'Loading model from {path}')
    model = tf.keras.models.load_model(path)
    model_1_list.append(model)
    print(f'Model from {path} loaded successfully')

for path in model2_paths:
    print(f'Loading model from {path}')
    model = tf.keras.models.load_model(path)
    model_2_list.append(model)
    print(f'Model from {path} loaded successfully')

Loading model from /Users/eding/PycharmProjects/U24-ROBOKOP-Project-8-21-24/Step 3 External Validation/Classification Models/Complete Set 1 Models/ROBOMechDB Complete Set 1 Model Seed 1.keras
Model from /Users/eding/PycharmProjects/U24-ROBOKOP-Project-8-21-24/Step 3 External Validation/Classification Models/Complete Set 1 Models/ROBOMechDB Complete Set 1 Model Seed 1.keras loaded successfully
Loading model from /Users/eding/PycharmProjects/U24-ROBOKOP-Project-8-21-24/Step 3 External Validation/Classification Models/Complete Set 1 Models/ROBOMechDB Complete Set 1 Model Seed 2.keras
Model from /Users/eding/PycharmProjects/U24-ROBOKOP-Project-8-21-24/Step 3 External Validation/Classification Models/Complete Set 1 Models/ROBOMechDB Complete Set 1 Model Seed 2.keras loaded successfully
Loading model from /Users/eding/PycharmProjects/U24-ROBOKOP-Project-8-21-24/Step 3 External Validation/Classification Models/Complete Set 1 Models/ROBOMechDB Complete Set 1 Model Seed 3.keras
Model from /User

In [3]:
#loading triple embeddings from robokops kg
df = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()),'Step 2 Data Embedding/Data Mined Embedded Dataset/ROBOKOP Virtual Screening Set Embedded Dataset.csv'))
df = df.drop('Unnamed: 0', axis = 1)
df = df.drop_duplicates(subset = ['0','2','4'])

In [4]:
print(df)

                 0            1                                     2  \
0       vandetanib  CHEBI:49960  multiple endocrine neoplasia type 2a   
1        ponatinib  CHEBI:78543  multiple endocrine neoplasia type 2a   
2        sorafenib  CHEBI:50924  multiple endocrine neoplasia type 2a   
3      regorafenib  CHEBI:68647  multiple endocrine neoplasia type 2a   
4      ruxolitinib  CHEBI:66919  multiple endocrine neoplasia type 2a   
...            ...          ...                                   ...   
89475  midostaurin  CHEBI:63452                          liver cancer   
89478     axitinib  CHEBI:66910                          liver cancer   
89479   canertinib  CHEBI:61399                          liver cancer   
89482    diazoxide   CHEBI:4495                          liver cancer   
89483  ceftriaxone  CHEBI:29007                          liver cancer   

                   3                                             4       5  \
0       DOID:0050430                         

In [5]:
data = df.iloc[:, 6:-1]

print(data[:1])


         6        7        8        9       10       11        12       13  \
0  0.79093  0.60086 -0.28079  0.40022 -0.50199  0.65795  0.013582 -0.07678   

         14       15  ...      596     597      598      599       600  \
0 -0.020119 -0.31451  ... -0.27445 -0.5197 -0.23558  0.54809  0.650776   

        601       602      603      604      605  
0 -0.738723  0.183145 -0.23919  0.72017 -0.22782  

[1 rows x 600 columns]


In [6]:
model_1_prediction_matrix = []
model_2_prediction_matrix = []

for j in range(len(model_1_list)):
    model_1_prediction_value = model_1_list[j].predict(data)
    model_1_prediction_value = model_1_prediction_value.flatten()
    model_1_prediction_matrix.append(model_1_prediction_value)
    
    model_2_prediction_value = model_2_list[j].predict(data)
    model_2_prediction_value = model_2_prediction_value.flatten()
    model_2_prediction_matrix.append(model_2_prediction_value)

model_1_prediction_matrix = np.array(model_1_prediction_matrix)
model_2_prediction_matrix = np.array(model_2_prediction_matrix)
    
model_1_stability_matrix = model_1_prediction_matrix.T
model_2_stability_matrix = model_2_prediction_matrix.T

model_1_prediction_floats = np.array([np.mean(row) for row in model_1_stability_matrix])
model_2_prediction_floats = np.array([np.mean(row) for row in model_2_stability_matrix])
model_prediction_floats = 0.9*model_1_prediction_floats + 0.1*model_2_prediction_floats

data_rounded = np.round(model_prediction_floats)
descending_confidence_values = np.sort(model_prediction_floats.flatten())[::-1]
ascending_indices = np.argsort(model_prediction_floats.flatten())
descending_indices = np.flip(ascending_indices)

1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 825us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 864us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 884us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 850us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 891us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 832us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 776us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 839us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 828us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 886us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 815us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 828us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 796us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 841us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 761us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 859us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 837us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 806us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 784us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 826us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 786us/step
1482/1482 ━━━━━━━━━━━━━━━━━━━━ 1s 862us/step
1482/1482 

In [7]:
print(descending_confidence_values)
print(descending_indices)

[0.9753456  0.97500587 0.97497606 ... 0.01408539 0.01321646 0.01236172]
[  993 44262 44271 ... 16756 19534  6653]


In [8]:
## finding the indices of only the 1's (actives)

true_positive_indices = []
true_positive_count = 0

false_negative_indices = []
false_negative_count = 0

for i in range (0,len(data_rounded)):
    if data_rounded[i] == 1:
        true_positive_indices.append(i)
        true_positive_count += 1
    else:
        false_negative_indices.append(i)
        false_negative_count +=1

print(true_positive_indices[:20])
print(true_positive_count)
print(false_negative_indices[:20])
print(false_negative_count)

[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 14, 15, 16, 20, 21, 22, 23, 24, 28]
20738
[6, 12, 13, 17, 18, 19, 25, 26, 27, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40]
26660


In [9]:
prediction_array = []
for i in descending_indices:
    triple_name = df.iloc[i,:6].tolist()
    conf_value = float(model_prediction_floats[i])
    row = [triple_name,conf_value]
    flat_row = row[0]+[row[1]]
    prediction_array.append(flat_row)



In [10]:
print(prediction_array[:2])

[['hydrocortisone cypionate', 'CHEBI:5783', "hodgkin's granuloma", 'DOID:8651', 'nuclear receptor subfamily 3 group c member 1', 'P04150', 0.9753456115722656], ['risperidone', 'CHEBI:8871', 'substance-related disorder', 'MONDO:0002494', 'dopamine receptor d2', 'P14416', 0.9750058650970459]]


In [12]:
columns_to_use = ['drug_name','drug_id','disease_name','disease_id','protein_name','protein_id','model_confidence_value']
df = pd.DataFrame(data = prediction_array,columns = columns_to_use)
print(df)
df.to_csv(os.path.join(os.getcwd(),'Screening Results/Bag of Ensembles Predictions of Top ROBOKOP Virtual Screening Set Triples.csv'))

                      drug_name       drug_id                disease_name  \
0      hydrocortisone cypionate    CHEBI:5783         hodgkin's granuloma   
1                   risperidone    CHEBI:8871  substance-related disorder   
2                  fluphenazine    CHEBI:5123  substance-related disorder   
3                     spiperone    CHEBI:9233   major depressive disorder   
4                   haloperidol    CHEBI:5613  substance-related disorder   
...                         ...           ...                         ...   
47393              dexibuprofen   CHEBI:43415                      ptosis   
47394              pimavanserin  CHEBI:133017         chiari malformation   
47395                 cp-724714  CHEBI:167658                      ptosis   
47396               doramapimod   CHEBI:40953            blepharophimosis   
47397              dexibuprofen   CHEBI:43415            thrombocytopenia   

          disease_id                                   protein_name  \
0   

In [3]:
df_mined = pd.read_csv(os.path.join(os.getcwd(),'Screening Results/Bag of Ensembles Predictions of Top ROBOKOP Virtual Screening Set Triples.csv'))
df_mined.drop(['Unnamed: 0'], axis = 1, inplace = True)



In [20]:
print(len(df_mined))

47398
